In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
imp = IterativeImputer(max_iter=10, random_state=0)
df = pd.read_csv('/content/drive/MyDrive/DALI/Superstore2.csv', index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Imputation with Scikit-learn Multivariate Imputation

In [ ]:
#divide into half being imputed based on training from the other half- these will form the training/testing sets
X, Xtest  = train_test_split(df[['Customer ID','Product ID','Postal Code','Sales','Quantity','Discount','Profit']].to_numpy(), test_size = 0.5)
imp.fit(X)
data = imp.transform(X_test)
dfX = pd.DataFrame(data, columns = ['CustomerID','ProductID','Postal Code','Sales','Quantity','Discount','Profit'])

missingIdx = dfX[pd.isna(dfX['Profit'])].index
dfX.drop(missingIdx, inplace=True)

In [ ]:
dfX

,CustomerID,ProductID,Postal Code,Sales,Quantity,Discount,Profit
0,705.0,404.0,37620.000000,170.352000,3.000000,0.2,10.6470
1,542.0,1183.0,10035.000000,321.568000,2.000000,0.2,-16.0784
2,253.0,975.0,63116.000000,52.590000,3.000000,0.0,15.7770
3,247.0,1664.0,80219.000000,6.800000,3.679964,0.2,0.5100
4,143.0,1730.0,28027.000000,315.776000,3.819460,0.2,31.5776
...,...,...,...,...,...,...,...
4861,627.0,621.0,28205.000000,309.169497,5.000000,0.2,26.4285
4862,552.0,343.0,63116.000000,111.790000,7.000000,0.0,43.5981
4863,669.0,350.0,10024.000000,45.576000,3.000000,0.2,15.9516
4864,35.0,839.0,43229.000000,419.400000,5.000000,0.2,146.7900


SVD Algorithm with Surprise Framework

In [ ]:
!pip install recommenders
import sys
import surprise

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse,
    mae,
)
from recommenders.models.surprise.surprise_utils import (
    predict,
    compute_ranking_predictions,
)
from recommenders.utils.notebook_utils import store_metadata
from surprise import Dataset, Reader

In [ ]:
train = pd.DataFrame(X, columns = ['CustomerID','ProductID','Postal Code','Sales','Quantity','Discount','Profit'])
train = train[['CustomerID','ProductID','Profit']]
pd.DataFrame(train.values).to_csv('/content/drive/MyDrive/DALI/train.csv')
test = dfX[['CustomerID','ProductID','Profit']]
reader = Reader(line_format='user item rating',sep=',',rating_scale=(-10000,10000))
#bound on the range of profit as a conservative scale
train_set = surprise.Dataset.load_from_file(
    '/content/drive/MyDrive/DALI/train.csv', reader
).build_full_trainset()

In [ ]:
svd = surprise.SVD(random_state=0, n_factors=16, n_epochs=30, verbose=True)
svd.fit(train_set)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29


In [ ]:
predictions = predict(svd, test, usercol='CustomerID', itemcol='ProductID')
predictions
#why are they all the same?

,CustomerID,ProductID,prediction
0,705.0,404.0,796.59926
1,542.0,1183.0,796.59926
2,253.0,975.0,796.59926
3,247.0,1664.0,796.59926
4,143.0,1730.0,796.59926
...,...,...,...
4861,627.0,621.0,796.59926
4862,552.0,343.0,796.59926
4863,669.0,350.0,796.59926
4864,35.0,839.0,796.59926


In [ ]:
test

,CustomerID,ProductID,Profit
0,705.0,404.0,10.6470
1,542.0,1183.0,-16.0784
2,253.0,975.0,15.7770
3,247.0,1664.0,0.5100
4,143.0,1730.0,31.5776
...,...,...,...
4861,627.0,621.0,26.4285
4862,552.0,343.0,43.5981
4863,669.0,350.0,15.9516
4864,35.0,839.0,146.7900


In [ ]:
eval_rmse = rmse(test, predictions)
eval_mae = mae(test, predictions)
print(eval_rmse)
print(eval_mae)
#predict function is not well documented- bug unsolved in time

ColumnMismatchError: Missing columns in true rating DataFrame